In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import os
import json
import io
import pandas as pd

In [67]:
import sys

sys.path.append("~/ep-backend/spai-new/spai")
sys.path

['/Users/judith/micromamba/envs/parquet/lib/python38.zip',
 '/Users/judith/micromamba/envs/parquet/lib/python3.8',
 '/Users/judith/micromamba/envs/parquet/lib/python3.8/lib-dynload',
 '/Users/judith/micromamba/envs/parquet/lib/python3.8/site-packages',
 '~/ep-backend/spai-new/spai']

In [192]:
from spai.spai.storage import Storage

In [193]:
os.environ["SPAI_STORAGE_NAMES"] = "local_data,s3_minio"
os.environ["SPAI_STORAGE_LOCAL_DATA_PATH"] = "data"
os.environ["SPAI_STORAGE_S3_MINIO_URL"] = "localhost:9000"
os.environ["SPAI_STORAGE_S3_MINIO_ACCESS"] = "spai"
os.environ["SPAI_STORAGE_S3_MINIO_SECRET"] = "12345678"
os.environ["SPAI_STORAGE_S3_MINIO_BUCKET"] = "devbucket"

In [194]:
# list env variables
!env | grep SPAI

SPAI_STORAGE_NAMES=local_data,s3_minio
SPAI_STORAGE_LOCAL_DATA_PATH=data
SPAI_STORAGE_S3_MINIO_URL=localhost:9000
SPAI_STORAGE_S3_MINIO_ACCESS=spai
SPAI_STORAGE_S3_MINIO_SECRET=12345678
SPAI_STORAGE_S3_MINIO_BUCKET=devbucket


In [195]:
storage = Storage()

'data' folder ready to use
'devbucket' bucket ready to use


In [216]:
data, minio = storage["data"], storage["minio"]
data.list()

['Weather.parquet', 'Weather2.parquet', 'growth.json']

In [217]:
minio.list()

['tif/S2L2A_2024-01-20.tif']

### Read & Save .parquet files in Local


In [209]:
df = data.read("Weather.parquet")
df

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,3.4,6.3,NW,30,SW,NW,6,...,29,1019.7,1015.0,7,7,14.4,23.6,No,3.6,Yes
1,14.0,26.9,3.6,4.4,9.7,ENE,39,E,W,4,...,36,1012.4,1008.4,5,3,17.5,25.7,Yes,3.6,Yes
2,13.7,23.4,3.6,5.8,3.3,NW,85,N,NNE,6,...,69,1009.5,1007.2,8,7,15.4,20.2,Yes,39.8,Yes
3,13.3,15.5,39.8,7.2,9.1,NW,54,WNW,W,30,...,56,1005.5,1007.0,2,7,13.5,14.1,Yes,2.8,Yes
4,7.6,16.1,2.8,5.6,10.6,SSE,50,SSE,ESE,20,...,49,1018.3,1018.5,7,7,11.1,15.4,Yes,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,9.0,30.7,0.0,7.6,12.1,NNW,76,SSE,NW,7,...,15,1016.1,1010.8,1,3,20.4,30.0,No,0.0,No
362,7.1,28.4,0.0,11.6,12.7,N,48,NNW,NNW,2,...,22,1020.0,1016.9,0,1,17.2,28.2,No,0.0,No
363,12.5,19.9,0.0,8.4,5.3,ESE,43,ENE,ENE,11,...,47,1024.0,1022.8,3,2,14.5,18.3,No,0.0,No
364,12.5,26.9,0.0,5.0,7.1,NW,46,SSW,WNW,6,...,39,1021.0,1016.2,6,7,15.8,25.9,No,0.0,No


In [210]:
data.create(df, "Weather2.parquet")

'data/Weather2.parquet'

In [211]:
data.list()

['Weather.parquet', 'Weather2.parquet', 'growth.json']

## Read & Save .parquet files in Cloud


In [219]:
s3_client = minio.client
s3_client.list_buckets()

[Bucket('devbucket')]

In [224]:
import s3fs
from pyarrow import Table, parquet as pq

fs = s3fs.S3FileSystem(
    anon=False,
    # key=s3fs_config.aws_access_key_id,
    # secret=s3fs_config.aws_secret_access_key,
    use_ssl=False,
    client_kwargs={
        "region_name": "us-east-1",
        "endpoint_url": "http://localhost:9000",
        "aws_access_key_id": "spai",
        "aws_secret_access_key": "12345678",
        "verify": False,
    },
)

path_to_s3_object = "s3://devbucket/Weather.parquet"

pq.write_to_dataset(
    Table.from_pandas(df),
    path_to_s3_object,
    filesystem=fs,
    use_dictionary=True,
    compression="snappy",
    version="2.4",
)

In [251]:
# list bucket content
fs.ls("devbucket")
# list prefix content
fs.ls("devbucket/Weather.parquet")

['devbucket/Weather.parquet/9f64d8fc366f401ea96329f5e43ada6a-0.parquet',
 'devbucket/Weather.parquet/e6dd9d243e7b4d29832b514d58e8ba40-0.parquet']

In [255]:
import s3fs
import pyarrow.parquet as pq

fs = s3fs.S3FileSystem(
    anon=False,
    # key=s3fs_config.aws_access_key_id,
    # secret=s3fs_config.aws_secret_access_key,
    use_ssl=False,
    client_kwargs={
        "region_name": "us-east-1",
        "endpoint_url": "http://localhost:9000",
        "aws_access_key_id": "spai",
        "aws_secret_access_key": "12345678",
        "verify": False,
    },
)

content = fs.ls("devbucket/Weather.parquet")[-1]

path_to_s3_object = f"s3://{content}"
dataset = pq.ParquetDataset(path_to_s3_object, filesystem=fs)
table = dataset.read()
df = table.to_pandas()
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,3.4,6.3,NW,30,SW,NW,6,...,29,1019.7,1015.0,7,7,14.4,23.6,No,3.6,Yes
1,14.0,26.9,3.6,4.4,9.7,ENE,39,E,W,4,...,36,1012.4,1008.4,5,3,17.5,25.7,Yes,3.6,Yes
2,13.7,23.4,3.6,5.8,3.3,NW,85,N,NNE,6,...,69,1009.5,1007.2,8,7,15.4,20.2,Yes,39.8,Yes
3,13.3,15.5,39.8,7.2,9.1,NW,54,WNW,W,30,...,56,1005.5,1007.0,2,7,13.5,14.1,Yes,2.8,Yes
4,7.6,16.1,2.8,5.6,10.6,SSE,50,SSE,ESE,20,...,49,1018.3,1018.5,7,7,11.1,15.4,Yes,0.0,No


In [256]:
minio.read("Weather.parquet")

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,3.4,6.3,NW,30,SW,NW,6,...,29,1019.7,1015.0,7,7,14.4,23.6,No,3.6,Yes
1,14.0,26.9,3.6,4.4,9.7,ENE,39,E,W,4,...,36,1012.4,1008.4,5,3,17.5,25.7,Yes,3.6,Yes
2,13.7,23.4,3.6,5.8,3.3,NW,85,N,NNE,6,...,69,1009.5,1007.2,8,7,15.4,20.2,Yes,39.8,Yes
3,13.3,15.5,39.8,7.2,9.1,NW,54,WNW,W,30,...,56,1005.5,1007.0,2,7,13.5,14.1,Yes,2.8,Yes
4,7.6,16.1,2.8,5.6,10.6,SSE,50,SSE,ESE,20,...,49,1018.3,1018.5,7,7,11.1,15.4,Yes,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,9.0,30.7,0.0,7.6,12.1,NNW,76,SSE,NW,7,...,15,1016.1,1010.8,1,3,20.4,30.0,No,0.0,No
362,7.1,28.4,0.0,11.6,12.7,N,48,NNW,NNW,2,...,22,1020.0,1016.9,0,1,17.2,28.2,No,0.0,No
363,12.5,19.9,0.0,8.4,5.3,ESE,43,ENE,ENE,11,...,47,1024.0,1022.8,3,2,14.5,18.3,No,0.0,No
364,12.5,26.9,0.0,5.0,7.1,NW,46,SSW,WNW,6,...,39,1021.0,1016.2,6,7,15.8,25.9,No,0.0,No


# Storage CRUD Operations


In [218]:
s3_client = minio.client
s3_client.list_buckets()

[Bucket('devbucket')]

In [ ]:
import rasterio as rio

# Assuming s3_client is already configured and connected
try:
    # Get the object from devbucket
    response = s3_client.get_object("devbucket", "tif/S2L2A_2019-06-02.tif")

    # Write the file data to a local file
    with open("local_filename.tif", "wb") as file:
        for d in response.stream(32 * 1024):
            file.write(d)

    print("File downloaded successfully.")
finally:
    response.close()
    response.release_conn()

# Now, try to open the image with rasterio
try:
    with rio.open("local_filename.tif") as dataset:
        print(dataset.meta)
except rio.errors.RasterioIOError as e:
    print(e)

In [ ]:
try:
    with rio.open("data/S2L2A_2019-06-02.tif") as dataset:
        print(dataset.meta)
except rio.errors.RasterioIOError as e:
    print(e)

In [ ]:
s3_client.fput_object(
    "devbucket", "tif/S2L2A_2019-06-02.tif", "data/S2L2A_2019-06-02.tif"
)

In [ ]:
content_type = "image/tiff"
prefix = "tif/"
name = "S2L2A_2019-06-02.tif"
dst_path = f"{prefix}{name}"
image_path = "data/S2L2A_2019-06-02.tif"

# Open the TIFF file in binary mode and upload it to the MinIO bucket
with open(image_path, "rb") as image_file:
    file_stat = os.stat(image_path)
    # Upload the file using put_object
    s3_client.put_object(
        bucket_name="devbucket",
        object_name=dst_path,
        data=image_file,
        length=file_stat.st_size,  # size of the file to upload
        content_type=content_type,
    )

print(f"Uploaded TIFF image to {dst_path}")

## List Objects


In [6]:
data.list()

['verges.geojson',
 'verges_3.geojson',
 'growth.json',
 'S2L2A_2019-06-02.tif',
 'growth_2.json',
 'growth_3.json']

In [7]:
minio.list()

['geojson/verges.geojson',
 'geojson/verges_2.geojson',
 'geojson/verges_3.geojson',
 'geojson/verges_4.geojson',
 'image-object-name.tif',
 'json/config.json',
 'json/growth_2.json',
 'json/growth_3.json',
 'json/growth_4.json',
 'png/logo.png',
 'tif/S2L2A_2019-06-02.tif',
 'tif/S2L2A_2019-06-02_copy.tif',
 'tif/S2L2A_2024-01-07.tif']

## Create and Update Objects


JSON FROM PATH


In [8]:
data.create(data="source/growth.json", name="growth.json")

'data/growth.json'

In [9]:
minio.create(data="data/growth.json", name="growth.json")

'json/growth.json'

JSON FROM DICT


In [10]:
growth = {
    "Not Vegetation Ha": {
        "1686873600000": 2172.52,
        "1686873600000": 2172.52,
        "1687737600000": 2095.39,
    },
    "Vegetation Ha": {
        "1686873600000": 316.05,
        "1686873600000": 316.05,
        "1687737600000": 393.18,
    },
    "Total": {
        "1686873600000": 2488.57,
        "1686873600000": 2488.57,
        "1687737600000": 2488.57,
    },
}

In [11]:
data.create(data=growth, name="growth_2.json")

'data/growth_2.json'

In [12]:
minio.create(data=growth, name="growth_2.json")

'json/growth_2.json'

JSON FROM DATAFRAME


In [13]:
df = pd.DataFrame(growth)

In [14]:
data.create(data=df, name="growth_3.json")

'data/growth_3.json'

In [15]:
minio.create(data=df, name="growth_3.json")

'json/growth_3.json'

GEOJSON FROM PATH


In [17]:
data.create(data="source/verges.geojson", name="verges.geojson")

'data/verges.geojson'

In [18]:
minio.create(data="data/verges.geojson", name="verges.geojson")

'geojson/verges.geojson'

GEOJSON FROM DICT


In [19]:
verges = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [3.0439, 42.076056],
                        [3.017464, 42.052607],
                        [3.0439, 42.032209],
                        [3.079262, 42.055156],
                        [3.0439, 42.076056],
                    ]
                ],
            },
        }
    ],
}

In [20]:
data.create(data=verges, name="verges_2.geojson")

'data/verges_2.geojson'

In [21]:
minio.create(data=verges, name="verges_2.geojson")

'geojson/verges_2.geojson'

TIF FROM PATH


In [22]:
data.create(data="source/S2L2A_2019-06-02.tif", name="S2L2A_2019-06-02.tif")

'data/S2L2A_2019-06-02.tif'

In [ ]:
minio.create(data="data/S2L2A_2019-06-02.tif", name="S2L2A_2019-06-02_copy.tif")

## Retrieve Objects


READ JSON


In [24]:
data.read("growth.json")

,Not Vegetation Ha,Vegetation Ha,Total
2023-06-16,2172.52,316.05,2488.57
2023-06-26,2095.39,393.18,2488.57


In [25]:
minio.read("growth.json")

,Not Vegetation Ha,Vegetation Ha,Total
2023-06-16,2172.52,316.05,2488.57
2023-06-26,2095.39,393.18,2488.57


READ GEOJSON


In [26]:
data.read("verges.geojson")

,geometry
0,"POLYGON ((3.04390 42.07606, 3.01746 42.05261, ..."


In [27]:
minio.read("verges.geojson")

,geometry
0,"POLYGON ((3.04390 42.07606, 3.01746 42.05261, ..."


READ TIF


In [28]:
data.read("S2L2A_2019-06-02.tif")

<open DatasetReader name='data/S2L2A_2019-06-02.tif' mode='r'>

In [ ]:
minio.read("S2L2A_2019-06-02.tif")